# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

## 필요한 library 호출

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2

import os

tf.logging.set_verbosity(tf.logging.INFO)


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## CNN model 설계
현재는 MNIST에 사용된 것으로, 추후 그물 찢김에 적용하기 위해 수정 필요

### Input Layer
Input Image Pixel: 28x28  
Color channel: 1 (grayscale)  

### 1st Conv Layer
Num of filter: 32  
Kernel size: 5x5  
Padding: Same  
Activation: ReLU  
Padding same은 입/출력이 같은 크기를 갖도록 padding하는 것  

### 1st Pooling Layer
Max pooling  
Size: 2x2  
Stride: 2  
Max pooling 시 size의 폭과 stride가 같으면 pooling window가 서로 겹치지 않음  

### 2nd Conv Layer
Num of filter: 64  
Kernel size: 5x5  
Padding: Same  
Activation: ReLU  

### 2nd Pooling Layer
Max pooling  
Size: 2x2  
Stride: 2  


### 3rd, 4th, 5th


### Dense Layer
Fully connected  

### Drop out
Rate: 0.4 (40%)  
훈련 중 40%의 connection이 임의로 제거됨  
Overfitting 방지  
뇌의학에서도 학습을 반복할수록 사람 뇌의 뉴런은 일부 connection이 끊어지면서 더 확실하게 학습한다고 함  

### Logits Layer
Binaray classification  

### Training
Loss function: Softmax Cross Entropy  
Optimizer: Gradient Descent  
Learning rate: 0.001  

In [2]:
def cnn_model_fn(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, height, width, channels]
    # Our Fishing net image size is 640x480 and 3-channel (RGB)
    input_layer = tf.reshape(features["x"], [-1, 480, 640, 3])

    # Convolutional Layer #1
    # Computes 48 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 480, 640, 3]
    # Output Tensor Shape: [batch_size, 480, 640, 48]
    conv1 = tf.layers.conv2d(
        
        inputs=input_layer,
        filters=48,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 480, 640, 48]
    # Output Tensor Shape: [batch_size, 240, 320, 48]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 96 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 240, 320, 48]
    # Output Tensor Shape: [batch_size, 240, 320, 96]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 240, 320, 96]
    # Output Tensor Shape: [batch_size, 120, 160, 96]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    
    
    
    # Input Tensor Shape: [batch_size, 120, 160, 96]
    # Output Tensor Shape: [batch_size, 120, 160, 96]    
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    # Input Tensor Shape: [batch_size, 120, 160, 96]
    # Output Tensor Shape: [batch_size, 60, 80, 96]
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    
    
    
    # Input Tensor Shape: [batch_size, 60, 80, 96]
    # Output Tensor Shape: [batch_size, 60, 80, 96]    
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    # Input Tensor Shape: [batch_size, 60, 80, 96]
    # Output Tensor Shape: [batch_size, 30, 40, 96]
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    
    
    
    # Input Tensor Shape: [batch_size, 30, 40, 96]
    # Output Tensor Shape: [batch_size, 30, 40, 96]    
    conv5 = tf.layers.conv2d(
        inputs=pool4,
        filters=96,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    # Input Tensor Shape: [batch_size, 30, 40, 96]
    # Output Tensor Shape: [batch_size, 15, 20, 96]
    pool5 = tf.layers.max_pooling2d(inputs=conv5, pool_size=[2, 2], strides=2)
    
    
    
    
    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 15, 20, 96]
    # Output Tensor Shape: [batch_size, 15 * 20 * 96]
    pool5_flat = tf.reshape(pool5, [-1, 15 * 20 * 96])


    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 15 * 20 * 96]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool5_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 2]
    logits = tf.layers.dense(inputs=dropout, units=2)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## Code Run


### Training
Batch size: 128  
Epoch: 5  
Shuffle: True  
Step: 20000

## Loading dataset
Load dataset and split them into training data and evaluation data

In [3]:
# torn: 0-1484
# untorn: 0-1779
num_torn = 1485
num_untorn = 1780
total_pix = 480*640*3

torn_data = np.zeros((num_torn, total_pix), dtype=np.float32)
print(torn_data.shape)
untorn_data = np.zeros((num_untorn, total_pix), dtype=np.float32)

(1485, 921600)


In [4]:
torn_idx = np.arange(num_torn)
untorn_idx = np.arange(num_untorn)
print(torn_idx)

np.random.shuffle(torn_idx)
np.random.shuffle(untorn_idx)

print(torn_idx)

[   0    1    2 ... 1482 1483 1484]
[1252   61 1193 ... 1045   22 1315]


In [5]:
for i in range(num_torn):
    img = cv2.imread("/dataset/torn/img-%04d.png" % torn_idx[i]).flatten()
    torn_data[i][:] = img

In [6]:
for i in range(num_untorn):
    img = cv2.imread("/dataset/untorn/img-%04d.png" % untorn_idx[i]).flatten()
    untorn_data[i][:] = img

In [7]:
torn_label = np.full((num_torn), 1)
untorn_label = np.full((num_untorn), 1)

In [8]:
# 80% training, 20% evaluation
tr_rate = 0.8
num_tr_torn = int(tr_rate * num_torn)
num_ev_torn = num_torn - num_tr_torn
print(num_tr_torn, num_ev_torn)

num_tr_untorn = int(tr_rate * num_untorn)
num_ev_untorn = num_untorn - num_tr_untorn
print(num_tr_untorn, num_ev_untorn)

tr_torn = torn_data[:num_tr_torn][:]
ev_torn = torn_data[num_tr_torn:][:]
print(tr_torn.shape)
print(ev_torn.shape)

tr_untorn = untorn_data[:num_tr_untorn][:]
ev_untorn = untorn_data[num_tr_untorn:][:]
print(tr_untorn.shape)
print(ev_untorn.shape)



tr_data = np.append(tr_torn, tr_untorn, axis=0)
print(tr_data.shape)
ev_data = np.append(ev_torn, ev_untorn, axis=0)
print(ev_data.shape)

tr_label = np.append(np.full((num_tr_torn), 1), np.full((num_tr_untorn), 0))
ev_label = np.append(np.full((num_ev_torn), 1), np.full((num_ev_untorn), 0))
print(tr_label.shape, ev_label.shape)

1188 297
1424 356
(1188, 921600)
(297, 921600)
(1424, 921600)
(356, 921600)
(2612, 921600)
(653, 921600)
(2612,) (653,)


In [9]:
shuf1 = np.arange(tr_label.shape[0])
np.random.shuffle(shuf1)
print(shuf1)

shuf2 = np.arange(ev_label.shape[0])
np.random.shuffle(shuf2)
print(shuf2)

tr_data_sh = np.zeros((tr_data.shape))
tr_label_sh = np.zeros(tr_label.shape)
ev_data_sh = np.zeros((ev_data.shape))
ev_label_sh = np.zeros(ev_label.shape)


for i in range(tr_label.shape[0]):
    tr_data_sh[i][:] = tr_data[shuf1[i]][:]
    tr_label_sh[i] = tr_label[shuf1[i]]
    
for i in range (ev_label.shape[0]):
    ev_data_sh[i][:] = ev_data[shuf2[i]][:]
    ev_label_sh[i] = ev_label[shuf2[i]]

[2351  317 1029 ... 1575 1292 2095]
[181  36 376  50 255 458 127 414  63 354 501 489  34 603 355 461 601 191
 428 466  82 157 554 233 288 532 300 607 454 278 542  84 120 403 291  69
 245 317 543 509 172 585 263 390 502 100 222 379 121 650 318 500 307 518
 319 452 151 621 487  98 112 624 211 104 131  18 484 192 604   0 371 456
 576 507 444  47 577 462 465  16 173  59 400 583 335 243 228 262 306 178
 602 348 584 269  27 283 508  83 159 606 260 476 569 528 572 593 386 470
 301 435 618   1 423 614 586 164 563 486 649 313 494 519 639  95 179 289
 387 365 516 514 550 384 504 616 123 627 137 524 592 409 449 579 236  49
 239 512 399 168 346 336 496 405 587 375 499 474 582 225 270  74 312 615
 642  31  37 451 374 571 421 480 339 149 410 258 578 105 625 332 275 148
 333 651 114 218 316 189 530 154 101 293 261 600 341 337 327 176 287 279
  58 256 364 205 595  20 102 589 297 455 580 111 640 152 366 193 344 221
 469 204 315 326  39  76  87 129 437 590 412 338 146 184 630 285  43 360
 636 210 169 29

In [10]:
print(tr_label_sh)

[0. 1. 1. ... 0. 0. 0.]


In [11]:
# GPU memory option
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4


# Create the Estimator
net_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="./models/")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

batch_s = 16
epochs = 5
tr_steps = int(tr_data.shape[0]/batch_s*epochs)
in_steps = 1

for i in range(tr_steps):
    tr_idx = np.arange(tr_data.shape[0])
    batch_picks = np.random.choice(tr_idx, batch_s, replace=False)
    
    inputs = np.zeros((batch_s, total_pix), dtype=np.float32)
    outputs = np.zeros(batch_s, dtype=np.int32)
    
    for j in range(batch_s):
        inputs[j][:] = tr_data_sh[batch_picks[j]][:]
        outputs[j] = tr_label_sh[batch_picks[j]]
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": inputs},
        y=outputs,
        batch_size=batch_s,
        num_epochs=1,
        shuffle=True)
    net_classifier.train(
        input_fn=train_input_fn,
        steps=in_steps,
        hooks=[logging_hook])


# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": ev_data},
    y=ev_label,
    num_epochs=1,
    shuffle=True)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_task_type': 'worker', '_master': '', '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_evaluation_master': '', '_tf_random_seed': None, '_task_id': 0, '_global_id_in_cluster': 0, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_keep_checkpoint_max': 5, '_model_dir': './models/', '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f94904fbe80>, '_save_summary_steps': 100, '_num_worker_replicas': 1, '_service': None, '_session_config': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./models/model.ckpt.
INFO:tensorflow:probabilities = [[0.         1.        ]
 [0.00001013 

NanLossDuringTrainingError: NaN loss during training.